## Bitboard für 4-Connect



In [ ]:
import numpy as np

#### Board

Ein *board* ist ein 6x7 numpy-Array mit den Werten:
```
0 - frei
1 - Spieler 1
2 - Spieler 2

```
 

Funktionen zum Erstellen und Füllen eines boards:

In [100]:
def makeBoard():
    return np.zeros((6, 7), dtype = int)

def showBoard(board):
    for i in range(6):
        print(*row(i,board))
    print('==============')
    print('0 1 2 3 4 5 6 ')
    print()
    
def row(i, board):
    return board[i,:]

def col(j, board):
    return board[:,j]

    
def insert(j, x, board):
    ''' insert in col j value x as in connect 4'''
    coltmp = np.argwhere(col(j,board)==0)  # 
    if len(coltmp) == 0: return False
    board[coltmp[-1],j] = x
    return True

def fill(j, s, board):
    ''' fill col j with values of s 
    eg. s = '1122' : 2 is at the bottom of col j '''
    for c in list(s)[::-1]:
        insert(j,int(c),board)

Beispiel:

In [101]:
board = makeBoard()
fill(2,'2111', board)
fill(3,'12212', board)
fill(4,'22', board)
fill(5,'2', board)
showBoard(board)

0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 2 0 0 0
0 0 1 2 0 0 0
0 0 1 1 2 0 0
0 0 1 2 2 2 0
0 1 2 3 4 5 6 



#### Bitboard, Boardzahl und Bitstring

Einem Board ordnen wir zwei Bitboards zu, die die Positionen der Steine jedes Spielers zeigen. 
Ein Bitboard ist eine 7x7 Matrix. Die erste Reihe ist eine Sentinel-Reihe (Wächter), die immer mit 0 gefüllt ist.

0 ist das *least significant bit*, d.h. das am weitesten rechts stehende Bit. Die Positionen der Bits in der Matrix:

In [17]:
bd = np.zeros((7, 7), dtype = int)

for j in range(7):
    for i in range(6,-1,-1):
        bd[i,j] = 6-i + 7 *j

for i in range(7):
    print(row(i,bd))
    if i == 0:
        print('----------------------')

[ 6 13 20 27 34 41 48]
----------------------
[ 5 12 19 26 33 40 47]
[ 4 11 18 25 32 39 46]
[ 3 10 17 24 31 38 45]
[ 2  9 16 23 30 37 44]
[ 1  8 15 22 29 36 43]
[ 0  7 14 21 28 35 42]


Das Bitboard wird nicht in einer Matrix gespeichert, sondern in einem *bitString*. Dieser Bitstring ergibt sich aus der Dualzahldarstellung einer Zahl. Diese Zahl nennen wir die boardZahl. Die Funktion *boardzahl* gibt neben der boardzahl für den Spieler *player* auch die Zahl
für die bitboard-Maske zurück. Die ist an allen Stellen 1, an denen ein Stein gesetzt wurde. Damit kann man das Bitboard des anderen
Spielers berechnen.

In [24]:
def boardzahl(board, player):
    ''' returns int : zahl, die im bin-format das board repräsentiert'''
    bs, mask = '', ''
    # Start with right-most column
    for j in range(6, -1, -1):
        # Add 0-bits to sentinel 
        mask += '0'
        bs += '0'
        # Start with bottom row
        for i in range(0, 6):
            mask += ['0', '1'][int(board[i, j] != 0)]
            bs += ['0', '1'][int(board[i, j] == player)]
    return int(bs, 2), int(mask, 2)

In [25]:
bz1, mask = boardzahl(board , 1)
bz2 = bz1 ^ mask  
bz1, bz2

(37863424, 35192438784)

In [102]:
board = makeBoard()
fill(2,'2111', board)
fill(3,'12212', board)
fill(4,'22', board)
fill(5,'1', board)
showBoard(board)

0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 2 0 0 0
0 0 1 2 0 0 0
0 0 1 1 2 0 0
0 0 1 2 2 1 0
0 1 2 3 4 5 6 



Wir betrachten den Bitstring, der durch die beiden Bitzahlen repräsentiert werden.

In [34]:
def bitboard(bz):
    bs = bitstr(bz)
    i, j = 6, 0
    board = np.zeros((7, 7), dtype = int)
    for c in bs[::-1]:
        board[i,j] = int(c)
        i-=1
        if i == -1:
            j += 1
            i = 6 
    print()
    for i in range(7):
        print(*row(i,board))
        if i == 0:
            print('-'*13)
    print('=============')
    print('0 1 2 3 4 5 6')

def bitstr(bz):
    bitstr = bin(bz)[2:]
    return bitstr.rjust(49,'0')

In [35]:
bitstr(bz1), bitstr(bz2)


('0000000000000000000000010010000011100000000000000',
 '0000000000000100000110001101000100000000000000000')

Aus den Bitstrings können wir das Bitboard konstruieren. Das Spiel-Board und die beiden Bitboards zusammen gesehen:

In [40]:
showBoard(board)
bitboard(bz1)
bitboard(bz2)

0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 2 0 0 0
0 0 1 2 0 0 0
0 0 1 1 2 0 0
0 0 1 2 2 2 0
0 1 2 3 4 5 6 


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 1 0 0 0
0 0 1 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 0 0 0
0 0 0 1 0 0 0
0 0 0 0 1 0 0
0 0 0 1 1 1 0
0 1 2 3 4 5 6


#### Check auf Vierer

Die Funktion entdeckt, ob vier Positionen verbunden sind

In [41]:
def check4(bz):
    '''  bn: bitnummer '''
    # Horizontal  
    m = bz & (bz >> 7)
    if m & (m >> 14):
        return True
    # Diagonal \
    m = bz & (bz >> 6)
    if m & (m >> 12):
        return True
    # Diagonal /
    m = bz & (bz >> 8)
    if m & (m >> 16):
        return True
    # Vertical
    m = bz & (bz >> 1)
    if m & (m >> 2):
        return True
    # Nothing found
    return False

In [42]:
check4(bz1), check4(bz2)

(False, True)

#### Erläuterung des horizontalen Checks

Das folgende Board dient zur Demonstration, es ist keinen gültige Spielstellung

In [45]:
board = makeBoard()
board[2] = np.array([0,0,1,1,1,1,0])    

bz = boardzahl(board,1)[0]
bitboard(bz)



0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 1 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Wir schieben die Bits in *bz* um 7 Stellen nach rechts.

In [46]:
bz1 = bz >> 7
print(bitstr(bz))
print(bitstr(bz1))

0000000000100000010000001000000100000000000000000
0000000000000000010000001000000100000010000000000


Das bedeutet, dass die Bits im Board um eine Stelle nach links verschoben wurden.

In [47]:
bitboard(bz)
bitboard(bz1)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 1 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 1 1 1 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Mit der *&*-Verbindung prüfen wir, ob an einer Stelle dasselbe steht, wie an der Stelle rechts daneben.

In [59]:
bitboard(m)
bitboard(m >> 14)
 


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 1 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Dieses Bitboard können wir wie folgt interpretieren: Dort wo eine 1 steht, steht im ursprünglichen bitboard rechts daneben auch eine 1.

Die beiden äußeren 1er verraten dass, ein 4er gefunden wurde. Die linke 1 sagt, hier steht eine 1 und rechts daneben auch. die rechte 1 sagt: hier steht eine 1 und rechts daneben auch. Von der linken 1 aus gesehen sind das vier 1en. 

Wir überprüfen, ob wir eine 1 und in der übernächsten Position wieder eine 1 haben, indem wir um 14 Stellen nach rechts verschieben und
wieder die *&*-Verbindung bilden. Wenn die resultierende boardzahl nicht gleich 0 ist, ist ein Vierer vorhanden

In [62]:
bitboard(m & m >> 14)
bool(m & m >> 14)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


True

#### Erläuterung des vertikalen Checks

Für den vertikalen Check benötigen wir die Sentinel-Reihe

#### Spielzug

In der Funktion *move* ist *bz* die Bitzahl des Spielers, der gerade dran ist. *bz1* ist dann die Bitzahl des anderen Spielers, die
unverändert zurückgegeben wird, da dieser Spieler das nächste mal dran ist und seine Steine sich in diesem Zug nicht verändern.

In [64]:
def move(bz, mask, col):
    bz1 = bz ^ mask
    mask1 = mask | (mask + (1 << (col*7)))
    return bz1, mask1

In [103]:
board = makeBoard()
fill(2,'211', board)
fill(3,'2', board)

showBoard(board)
bz, mask = boardzahl(board,1) 
bitboard(bz)

bz_neu, mask_neu = move(bz,mask,2)
bitboard(bz_neu)
bitboard(mask_neu)

0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 2 0 0 0 0
0 0 1 0 0 0 0
0 0 1 2 0 0 0
0 1 2 3 4 5 6 


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 0 1 1 0 0 0
0 1 2 3 4 5 6


Erläuterung zur neuen Maske

In [89]:
bitboard(mask)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 0 1 1 0 0 0
0 1 2 3 4 5 6


In [91]:
col = 2
bitboard(1)
bitboard(1 << (col*7))


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 0 0 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 1 2 3 4 5 6


In [92]:
bitboard(mask + (1 << (col*7)))


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 1 2 3 4 5 6


In [93]:
bitboard(mask | mask + (1 << (col*7)))


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 0 1 0 0 0 0
0 0 1 1 0 0 0
0 1 2 3 4 5 6


#### Überfüllen des Boards 
ist noch nicht abgefangen

In [107]:
board = makeBoard()
fill(2,'211', board)
fill(3,'2', board)
bz,mask = move(bz,mask,2)
bz,mask = move(bz,mask,2)
bz,mask = move(bz,mask,2)
bz,mask = move(bz,mask,2)
bitboard(mask)



0 0 1 0 0 0 0
-------------
0 0 1 1 0 0 0
0 0 1 1 0 0 0
0 0 1 1 0 0 0
0 0 1 1 0 0 0
0 0 1 1 0 0 0
0 0 1 1 0 0 0
0 1 2 3 4 5 6
